In [1]:
import numpy as np
import json
import pickle
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

In [2]:
## Loading word to index mapping
with open('./resources/word2idx.pkl','rb') as f:
    word2idx=pickle.load(f)

## Loading index to word mapping
with open('./resources/idx2word.pkl','rb') as f:
    idx2word=pickle.load(f)

## Loading ResNet50 testing image features     
with open("./resources/encoded_test_images.pkl","rb") as f:    
    encoded_test_images = pickle.load(f)

In [3]:
featureExtract_model = load_model("./resources/resNet50.h5")

def preprocess_image(img_path):
    img = image.load_img(img_path,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = preprocess_input(img)  ## preprocessing for ResNet50
    return img
def encode_image(img_path):
    img = preprocess_image(img_path)
    img_feature_vector = featureExtract_model.predict(img)
    img_feature_vector = img_feature_vector.reshape((-1,))
    return img_feature_vector

In [4]:
maxlen = 40
model = load_model('./models_weights/model.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [13]:
def predict_caption(image_path):
    
    photo = encode_image(image_path).reshape(1,2048)
    
    input_text = '<s>'
    for i in range(maxlen):
        sequence = [word2idx[word] for word in input_text.split() if word in word2idx]
        sequence = pad_sequences([sequence],maxlen=maxlen,padding='post')
        
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax()
        word = idx2word[ypred]
        
        input_text +=' ' + word
        
        if word == '<e>':
            break
            
    final_caption=input_text.split()[1:-1]
    final_caption=' '.join(final_caption)
    
    print(final_caption)
    img=plt.imread(PATH+image_name)
    plt.imshow(img)
    plt.show()

In [48]:
## pick and generating captions for random images from Test_Images

PATH = 'Test-Images/'
path = pathlib.Path(PATH)
folder = path.glob('*')

for img_path in list(folder)[:-1] :
    image_name = str(img_path).split('/')[1]
    predict_caption(PATH+image_name)

_To get a caption for an image call **predict_caption** by giving it path to that image._

In [15]:
#predict_caption(image_path)